### 背景建模

#### 定义
背景建模（Background Modeling）是一种**从视频序列中分离运动目标（前景）与静态场景（背景）**的技术。

#### 前提条件
输入要求：
1. 视频流或连续帧图像（时间相关性）
2. 场景光照相对稳定（剧烈变化需特殊处理）
3. 背景在大部分时间内可见（遮挡不超过50%帧数）

硬件/环境：
1. 固定摄像头视角（非移动拍摄）
2. 帧率稳定（避免运动模糊）

#### 核心原理
背景建模三要素：
1. 背景表示：建立背景的统计模型（如单帧/高斯混合/神经网络）
2. 前景检测：当前帧与背景模型差异超过阈值则判为前景
3. 模型更新：动态调整背景模型以摄影光照变化、鑫静态物体加入等场景

#### 常用方法
【下边介绍】

#### 应用场景
1. 智能监控：入侵检测、人流统计、异常行为分析
2. 交通监控：车辆检测、违章停车识别、交通流量统计
3. 工业检测：生产线物体运动追踪、缺陷检测
4. 视频会议：动态背景替换（虚拟背景）
5. 无人机导航：运动障碍物检测

#### 技术挑战与解决方案
1. 光照突变：使用自适应阈值或HSV色彩空间模型
2. 动态背景：选择MOG2/KNN等抗干扰算法
3. 阴影干扰：启用detectShadows=True参数过滤阴影
4. 长时间运动漂移：定期重置背景模型或设置学习参数


#### 性能优化技巧
1. 降低采样处理：对高清视频先缩小分辨率
2. ROI限制：旨在感兴趣区域运行算法
3. 多线程：分离背景建模和前景分析流程
4. 硬件加速：使用CUDA版本

### 帧差法

#### 核心原理
由于场景中的目标在运动，目标的影像在不同图像帧中的位置不同。该类算法对时间上连续的两帧图像进行差分运算，不同帧对应的像素点相减，判断灰度差的绝对值，当绝对值超过一定阈值时，即可判断为运动目标，从而实现目标的检测功能。


#### 关键特点
1. 计算速度快：仅需要像素级减法，适合实时处理
2. 无背景模型：直接比较相邻帧，无需建模
3. 对光照敏感：光照突变会导致误检
4. 空洞问题：运动目标内部区域可能检测不到（如纯色物体）

#### 优缺点
1. 实现简单，计算效率高，但无法处理静止目标（需多帧差分）
2. 无训练阶段，即使生效，但对噪声敏感（需配合滤波）
3. 内存占用低，但无法区分重叠运动目标


### 核心公式
1. $D_{n}(x,y) = |f_{n}(x,y)-f_{n-1}(x,y)| 计算前后两帧图像中的各个像素点像素值变化，保存差值$
2. $D_{n}(x,y) > T(阈值)，像素置为255(白色)，其余像素置为0(黑色)，白色的就是最终获取到的运动目标(前景)$

![title](bg_2.png)

帧差法非常简单，但是会引入噪音和空洞问题

### 混合高斯模型（MOG2）

在进行前景检测前，先对背景进行训练，对图像中每个背景采用一个混合高斯模型进行模拟，每个背景的混合高斯的个数可以自适应。然后在测试阶段，对新来的像素进行GMM匹配，如果该像素值能够匹配其中一个高斯，则认为是背景，否则认为是前景。由于整个过程GMM模型在不断更新学习中，所以对动态背景有一定的鲁棒性。最后通过对一个有树枝摇摆的动态背景进行前景检测，取得了较好的效果。


#### 核心原理
混合高斯模型通过**多个高斯分布**表示每个像素点的颜色值变化，适应复杂背景（如树叶摇动、水面波纹）。
1. 每个像素：用3~5个高斯分布建模
2. 分布参数：包括均值（颜色）、方差（变化浮度）、权重（出现频率）
3. 在线更新：根据新帧数据动态调整高斯分布参数

#### 参数调优建议
参数	     作用	                             推荐值
history	        影响模型记忆长度，值越大越稳定但响应越慢	200-1000
varThreshold	判定前景的方差阈值，值越小灵敏度越高	    10-25
detectShadows	是否检测阴影（True时阴影=127）	        True/False
shadowThreshold	阴影判定阈值（0-1，越小越容易判为阴影）	0.5（默认）

在视频中对于像素点的变化情况应当是符合高斯分布

![title](bg_3.png)

背景的实际分布应当是多个高斯分布混合在一起，每个高斯模型也可以带有权重

![title](bg_4.png)

#### 混合高斯模型学习方法

- 1.首先初始化每个高斯模型矩阵参数。

- 2.取视频中T帧数据图像用来训练高斯混合模型。来了第一个像素之后用它来当做第一个高斯分布。

- 3.当后面来的像素值时，与前面已有的高斯的均值比较，如果该像素点的值与其模型均值差在3倍的方差内，则属于该分布，并对其进行参数更新。

- 4.如果下一次来的像素不满足当前高斯分布，用它来创建一个新的高斯分布。


#### 混合高斯模型测试方法

在测试阶段，对新来像素点的值与混合高斯模型中的每一个均值进行比较，如果其差值在2倍的方差之间的话，则认为是背景，否则认为是前景。将前景赋值为255，背景赋值为0。这样就形成了一副前景二值图。

![title](bg_5.png)

In [ ]:
import cv2

# 经典的测试视频
# 视频捕获初始化
cap = cv2.VideoCapture('test.avi')

# 形态学操作需要使用
# 作用：创建3×3椭圆卷积核（用于后续形态学操作）
# 椭圆卷积核优势：比矩形卷积核更能保持物体圆形特征
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

# 创建混合高斯模型(MOG2)用于背景建模
# 默认参数：
# history=500：使用500帧建立别境模型
# varThreshold=16：像素方差阈值
# detectShadows=True：检测并标记阴影（灰色）
fgbg = cv2.createBackgroundSubtractorMOG2()

while(True):
    # 读取帧
    # 返回值：
    # ret：是否成功读取帧（False表示视频结束）
    # frame：当前帧的BGR图像（形状：[H,W,3]）
    ret, frame = cap.read()
    # 背景建模
    # 输出：
    # 前景区域：白色(255)
    # 背景区域：黑色(0)
    # 阴影区域：灰色(127，必须设定detectShadows=True这个参数才会展示)
    fgmask = fgbg.apply(frame)
    # 形态学开运算（先腐蚀[消除小噪点]，在膨胀[恢复主体形态]）去噪点
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    # 寻找视频中的轮廓
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # 计算各轮廓的周长
        perimeter = cv2.arcLength(c,True)
        # 根据实际业务，框住人体矩阵的轮廓大概周长在188
        if perimeter > 188:
            # 找到一个直矩形（不会旋转）
            # (x,y)：矩形左上角的点位
            # w：矩形宽度
            # h：矩形高度
            x,y,w,h = cv2.boundingRect(c)
            # 画出这个矩形（0,255.0）绿色
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

    # 展示原始视频图像
    cv2.imshow('frame',frame)
    # 展示提取到的前景图像
    cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()
